## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/bert-131-last/' + '*.h5')
model_path_list += glob.glob('/kaggle/input/bert-131-best/' + '*.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_1.h5
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_2.h5
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_3.h5
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_4.h5
/kaggle/input/bert-131-best/131-Bert_base_unc Quest_Ans3_model_fold_5.h5
/kaggle/input/bert-131-last/131-Bert_base_unc Quest_Ans3_model_fold_1_last_epoch.h5
/kaggle/input/bert-131-last/131-Bert_base_unc Quest_Ans3_model_fold_2_last_epoch.h5
/kaggle/input/bert-131-last/131-Bert_base_unc Quest_Ans3_model_fold_4_last_epoch.h5
/kaggle/input/bert-131-last/131-Bert_base_unc Quest_Ans3_model_fold_5_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = ['question_title', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512
MAX_LENGTH_TITLE = 30
MAX_LENGTH_QUESTION = 479
MAX_LENGTH_ANSWER = 479

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                               MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
        + compute_input_arays_2(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Model

In [7]:
def model_fn():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output_question, sequence_output_question = bert_layer([input_word_ids_question, input_masks_question, segment_ids_question])
    pooled_output_answer, sequence_output_answer = bert_layer([input_word_ids_answer, input_masks_answer, segment_ids_answer])

    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_question = Dropout(0.2)(x_question)
    output_question = Dense(N_CLASS_QUESTION, kernel_initializer='glorot_uniform', activation="sigmoid", name="outputquestion")(x_question)
    
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x_answer = Dropout(0.2)(x_answer)
    output_answer = Dense(N_CLASS_ANSWER, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_answer")(x_answer)
    
    output = Concatenate()([output_question, output_answer])

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)

    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.949419,0.678467,0.179049,0.525488,0.652020,0.535945,0.681226,0.609346,0.619791,...,0.910731,0.936471,0.577799,0.973427,0.976494,0.831899,0.027331,0.031273,0.791687,0.907355
1,46,0.838945,0.486215,0.009111,0.770087,0.754017,0.940375,0.527859,0.462307,0.099041,...,0.643131,0.934390,0.643464,0.970071,0.982491,0.866706,0.921981,0.127067,0.084553,0.879991
2,70,0.866436,0.679571,0.030185,0.785752,0.855923,0.916291,0.572685,0.474965,0.108544,...,0.853708,0.943492,0.577231,0.967011,0.964136,0.844097,0.035947,0.029184,0.839454,0.905079
3,132,0.865287,0.410151,0.009967,0.732915,0.719516,0.904731,0.511778,0.414774,0.094102,...,0.700080,0.951670,0.685368,0.973261,0.980499,0.903684,0.782473,0.158812,0.638241,0.893297
4,200,0.939248,0.502363,0.041433,0.866184,0.759104,0.904650,0.637982,0.599331,0.096904,...,0.671506,0.915947,0.641843,0.954278,0.961243,0.820752,0.268455,0.159911,0.543614,0.867158


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.881362,0.587248,0.034221,0.729795,0.806240,0.857231,0.564687,0.485909,0.233625,...,0.793293,0.931817,0.653049,0.963510,0.973043,0.866125,0.528872,0.138015,0.493018,0.906430
std,2812.670060,0.051691,0.145544,0.052807,0.105001,0.094207,0.120481,0.057398,0.090079,0.211157,...,0.093670,0.032080,0.049102,0.017022,0.014425,0.045167,0.321335,0.067478,0.286902,0.027645
min,39.000000,0.708502,0.334680,0.003741,0.290291,0.445926,0.262620,0.455186,0.342447,0.012798,...,0.554436,0.744987,0.469934,0.881271,0.868514,0.698075,0.010374,0.010672,0.024340,0.797311
25%,2572.000000,0.847481,0.457011,0.008980,0.671503,0.762298,0.837620,0.519631,0.416082,0.073611,...,0.713156,0.915214,0.621407,0.955986,0.966776,0.840144,0.195260,0.087702,0.237259,0.889751
50%,5093.000000,0.882571,0.567528,0.013770,0.735924,0.814067,0.904515,0.553789,0.458836,0.143763,...,0.790341,0.939725,0.650491,0.967552,0.977149,0.874975,0.603899,0.138116,0.489833,0.911493
75%,7482.000000,0.921330,0.716854,0.031069,0.795541,0.865613,0.930639,0.604567,0.534657,0.361219,...,0.881922,0.954678,0.682513,0.974830,0.982741,0.897078,0.828234,0.184572,0.751855,0.926293
max,9640.000000,0.976242,0.902538,0.403082,0.967775,0.975740,0.971354,0.731717,0.744405,0.859598,...,0.960698,0.984864,0.827858,0.988504,0.992412,0.955289,0.957314,0.362475,0.972791,0.957136
